# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import geoapify_key
#from api_keys import g_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("D:/MSU DATA/Module6_challenge/Starter_Code/Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
!pip install gmaps
!pip install jupyter_contrib_nbextensions

# Configure gmaps
gmaps.configure(api_key=geoapify_key)

# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[(city_data_df["Cloudiness"]<20) & (city_data_df["Humidity"]>0) &
                                (city_data_df["Max Temp"]>0) & (city_data_df["Max Temp"]<80)]
narrowed_city_df
# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna(how='any')


# Display sample data
#narrowed_city_df.reset_index(inplace=True)

#del narrowed_city_df['index']
narrowed_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
...,...,...,...,...,...,...,...,...,...,...
560,560,changli,39.7042,119.1506,9.17,33,0,2.41,CN,1666108557
564,564,redmond,47.6740,-122.1215,13.17,91,8,0.17,US,1666108556
565,565,wyndham,37.6985,-77.6125,12.81,52,3,1.79,US,1666108480
571,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,1666108563


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data

hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
29,saint-louis,SN,16.3333,-15.0000,24,
...,...,...,...,...,...,...
560,changli,CN,39.7042,119.1506,33,
564,redmond,US,47.6740,-122.1215,91,
565,wyndham,US,37.6985,-77.6125,52,
571,dargaville,NZ,-35.9333,173.8833,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": 10000,
    "types": "lodging",
    "key": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
  #  try:
        curr_lat = row["Lat"]
        curr_lng = row["Lng"]
  #  except (KeyError, IndexError):
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
        params["filter"] = f"{curr_lat},{curr_lng}"
        params["bias"] = f"{curr_lat},{curr_lng}"
    
    # Set base URL
        base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
        name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
        name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
         hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
margate - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
collierville - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
mogadishu - nearest hotel: No hotel found
hobart - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
lasa - nearest hotel: No hotel found
busselton - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
atuona - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
qostanay - nearest hotel: No hotel found
esperance - nearest hotel: No hotel found
solnechnyy - nearest hotel: No hote

,City,Country,Lat,Lng,Humidity,Hotel Name
1,farsund,NO,58.0948,6.8047,100,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,No hotel found
7,carnarvon,AU,-24.8667,113.6333,76,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
...,...,...,...,...,...,...
560,changli,CN,39.7042,119.1506,33,No hotel found
564,redmond,US,47.6740,-122.1215,91,No hotel found
565,wyndham,US,37.6985,-77.6125,52,No hotel found
571,dargaville,NZ,-35.9333,173.8833,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
# Configure gmaps
gmaps.configure(api_key=geoapify_key)

# Plot Heatmap
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig = gmaps.figure()


# Display the map
fig

Figure(layout=FigureLayout(height='420px'))